# knowledge graph index pdf pipeline

In [1]:
import os
import time

import openai
os.environ['OPENAI_API_KEY'] = "EMPTY"
os.environ['OPENAI_API_BASE'] = "http://10.0.0.222:30307/v1"
openai.api_key = "EMPTY"
openai.api_base = "http://10.0.0.222:30307/v1"

model = "Writer/camel-5b-hf"
#model = "mosaicml/mpt-7b-instruct"
#model = "mosaicml/mpt-30b-instruct"

In [2]:
import logging
import sys

#kron extensions to llama_index to support openai compatible api
sys.path.append('../llama_index')

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [3]:
from datetime import datetime
import json
import pathlib
import fitz

PDF_BASE = '/home/arylwen/datasets/documents/pdf/'
JSON_BASE = '/home/arylwen/datasets/documents/json_raw/'
doc_path = '/home/arylwen/datasets/documents/pdf/2007.02871.pdf'

In [4]:
document = fitz.open(doc_path)

In [5]:
document.metadata

{'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.21',
 'creationDate': 'D:20210413005313Z',
 'modDate': 'D:20210413005313Z',
 'trapped': '',
 'encryption': None}

In [6]:
if document.metadata['title'].strip() == '':
    pdf_name= doc_path.split('/')[-1]
    document.metadata['title'] = pdf_name


In [7]:
def get_pdf_content(document):
    content = ""
    for page in document:
        content += page.get_text()
    return content

print(get_pdf_content(document))

DART: Open-Domain Structured Data Record to Text Generation
Linyong Nan1
Dragomir Radev1,2
Rui Zhang3
Amrit Rau1
Abhinand Sivaprasad1
Chiachun Hsieh4
Xiangru Tang1
Aadit Vyas1
Neha Verma1
Pranav Krishna5
Yangxiaokang Liu1
Nadia Irwanto1
Jessica Pan1
Faiaz Rahman1
Ahmad Zaidi1
Murori Mutuma1
Yasin Tarabar1
Ankit Gupta1
Tao Yu1
Yi Chern Tan1
Xi Victoria Lin2∗
Caiming Xiong2
Richard Socher2
Nazneen Fatema Rajani2
1 Yale University
2 Salesforce Research
3 Penn State University
4 The University of Hong Kong
5 MIT
{linyong.nan, dragomir.radev}@yale.edu, rmz5227@psu.edu, nazneen.rajani@salesforce.com
Abstract
We present DART, an open domain structured
DAta Record to Text generation dataset with
over 82k instances (DARTs). Data-to-Text an-
notations can be a costly process, especially
when dealing with tables which are the ma-
jor source of structured data and contain non-
trivial structures. To this end, we propose a
procedure of extracting semantic triples from
tables that encodes their stru

In [8]:
def is_pymupdf_extractable(document,content):
    print(document.metadata['creator'])
    acceptable_creator = ['Springer','Pages', 'LaTeX with hyperref']
    words_threshold = 20
    if (document.metadata['creator'] in acceptable_creator) and (len(content) > words_threshold):
        return True
    else:
        return False
    
def write_json_file(filename, content):
    pathlib.Path(JSON_BASE+filename).write_bytes(content.encode('utf-8').strip())

def save_content(document,content):
    document_dict = dict()
    filename = document.metadata['title']+'.json'
    document_dict['title'] = document.metadata['title']
    document_dict['text'] = content
    document_dict['extraction_date'] = str(datetime.utcnow())
    document_dict['num_pages'] = document.page_count
    json_object = json.dumps(document_dict) 
    write_json_file(filename,json_object)

In [9]:
content = get_pdf_content(document)
if is_pymupdf_extractable(document,content):
    save_content(document,content)

LaTeX with hyperref
